In [1]:
# Importing Dependencies

In [3]:
import numpy as np
import pandas as pd
import re # It is used to clean text by removing non-letters, Removing Urls and space management.

from nltk.corpus import stopwords # Used to find stopwords such as ("the", "a", "is", "in").
from nltk.stem.porter import PorterStemmer # Used to find root word Ex-("Walking", "Walked", and "Walks" all become "Walk").
from sklearn.feature_extraction.text import TfidfVectorizer # Used to convert word into numbers to train model.
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ARTH\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [6]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [9]:
# Data Pre-Processing
# Importing Dataset

In [10]:
news_dataset = pd.read_csv('news.csv')

In [11]:
news_dataset.head(2)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0


In [12]:
# 1 == Fack news
# 0 == Real news

In [13]:
news_dataset.shape

(20800, 5)

In [17]:
# Find empty values in dataset

In [18]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [19]:
# Fill Null value with empty string

In [20]:
news_dataset = news_dataset.fillna('')

In [21]:
# Merging the auther name and news title

In [23]:
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title']

In [25]:
news_dataset['content']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [26]:
# seperating data and labels

In [27]:
X = news_dataset.drop(columns='label',axis=1)
Y = news_dataset['label']

In [28]:
X,Y

(          id                                              title  \
 0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
 1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
 2          2                  Why the Truth Might Get You Fired   
 3          3  15 Civilians Killed In Single US Airstrike Hav...   
 4          4  Iranian woman jailed for fictional unpublished...   
 ...      ...                                                ...   
 20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
 20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
 20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
 20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
 20799  20799                          What Keeps the F-35 Alive   
 
                                           author  \
 0                                  Darrell Lucus   
 1                                Daniel J. Flynn   
 2                     

In [29]:
# Stemming : Stemming is the process of reducing the word to its ROOT word

In [30]:
port_stem = PorterStemmer()

In [33]:
# Define stemming function
# 1 -- Uses Regular Expressions (re) to find anything that is not a letter (numbers, punctuation, emojis) and replaces it with a space.
# 2 -- Convert word to lower-case
# 3 -- Split the words
# 4 -- Remover all the stopwords
# 5 -- Stiches the all meaningful words
# 6 -- Return List of meaningful Words

In [34]:
def stemming(content):
    stem_content = re.sub('[^a-zA-Z]',' ',content)
    stem_content = stem_content.lower()
    stem_content = stem_content.split()
    stem_content = [port_stem.stem(word) for word in stem_content if not word in stopwords.words('english')]
    stem_content = ' '.join(stem_content)
    return stem_content

In [36]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [38]:
news_dataset['content']

0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exerci b...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [39]:
# Seperating Data and Labels

In [41]:
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [42]:
X,Y

(array(['darrel lucu hou dem aid even see comey letter jason chaffetz tweet',
        'daniel j flynn flynn hillari clinton big woman campu breitbart',
        'consortiumnew com truth might get fire', ...,
        'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
        'alex ansari nato russia hold parallel exerci balkan',
        'david swanson keep f aliv'], dtype=object),
 array([1, 0, 1, ..., 0, 1, 1]))

In [43]:
# Converting this text data to meaningful number to train model

In [44]:
vectorizer = TfidfVectorizer()

In [45]:
vectorizer.fit(X)
X = vectorizer.transform(X)

In [49]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 210584 stored elements and shape (20800, 16984)>
  Coords	Values
  (0, 263)	0.2701012497770876
  (0, 2464)	0.36765196867972083
  (0, 2933)	0.24684501285337127
  (0, 3567)	0.3598939188262558
  (0, 3759)	0.27053324808454915
  (0, 4917)	0.23331696690935097
  (0, 6930)	0.2187416908935914
  (0, 7612)	0.24785219520671598
  (0, 8546)	0.2921251408704368
  (0, 8822)	0.36359638063260746
  (0, 13348)	0.2565896679337956
  (0, 15553)	0.2848506356272864
  (1, 1481)	0.2957471154505952
  (1, 1877)	0.15614790568229528
  (1, 2206)	0.36915639258038363
  (1, 2790)	0.19208753385709676
  (1, 3535)	0.2653147533915268
  (1, 5440)	0.7186013955384664
  (1, 6744)	0.19152496072048605
  (1, 16656)	0.3025156488372128
  (2, 2917)	0.3179886800654691
  (2, 3072)	0.46097489583229645
  (2, 5326)	0.3866530551182615
  (2, 5903)	0.3474613386728292
  (2, 9532)	0.49351492943649944
  :	:
  (20797, 3610)	0.211655450844435
  (20797, 6969)	0.21809398920480086
  (20797

In [50]:
# Splitting the dataset into training and testing part

In [51]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=2, stratify=Y , test_size=0.2 )

In [53]:
# Train Model : Logistic Regression

In [54]:
model = LogisticRegression()

In [56]:
model.fit(X_train,Y_train)

LogisticRegression()

In [57]:
# Evaluation on training data

In [58]:
X_train_prediction = model.predict(X_train)

In [68]:
# Classification report and condusion metrics for training data

In [66]:
print('Training data Classification Report : \n',classification_report(X_train_prediction,Y_train))

Training data Classification Report : 
               precision    recall  f1-score   support

           0       0.98      0.99      0.99      8181
           1       0.99      0.98      0.99      8459

    accuracy                           0.99     16640
   macro avg       0.99      0.99      0.99     16640
weighted avg       0.99      0.99      0.99     16640



In [67]:
print('Training Data Confusion Metrics : \n',confusion_matrix(X_train_prediction,Y_train))

Training Data Confusion Metrics : 
 [[8132   49]
 [ 178 8281]]


In [64]:
# Evaluation on testing data

In [65]:
X_test_prediction = model.predict(X_test)

In [69]:
# Classification report and condusion metrics for testing data

In [70]:
print('Testing data Classification Report : \n',classification_report(X_test_prediction,Y_test))

Testing data Classification Report : 
               precision    recall  f1-score   support

           0       0.96      0.99      0.98      2018
           1       0.99      0.97      0.98      2142

    accuracy                           0.98      4160
   macro avg       0.98      0.98      0.98      4160
weighted avg       0.98      0.98      0.98      4160



In [71]:
print('Testing data Confusion Metrics : \n',confusion_matrix(X_test_prediction,Y_test))

Testing data Confusion Metrics : 
 [[2004   14]
 [  73 2069]]


In [73]:
# Make a predictive System

In [77]:
X_new = X_test.i[3]

prediction = model.predict(X_new)
print(prediction)

if(prediction == 0):
    print("The news is Real")
else:
    print("The news is Fake")

[0]
The news is Real
